# Convolutional Neral Network
Show how to build convolutional neral network with tensorflow higher-level API

## Biref theory
![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)


## Operations in CNN
Frequently used operations in CNN can be summarized below:
- Convolution: extract feature with convolution computation between kernel weights and image inputs
- Pooling:  very similar to how convolution operation works,  but using different computations, such as max operation.


# Dataset
This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)


# Emplementation
## Import modules

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
minist = input_data.read_data_sets("/tmp/data", one_hot=False)
                                  
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

URLError: <urlopen error [Errno 101] Network is unreachable>

## Parameters


In [ ]:
learning_rate = 0.001
num_steps = 2000
batch_size = 128

num_input = 784
n_classes = 10
dropout = 0.25

## Building nets
- network structure
- loss
- optimizer

In [ ]:
# Create network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    """
    x_dict: input in dicationary format 
    """
    with tf.variable_scope('ConvNet', reuse=reuse):
        x = x_dict['images']
        x = tf.reshape(x, shape=[-1, 28, 28, 1]) # reshpae into 2d image
        
        # Convolution layer with 32 filters and kenel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        
        fc1 = tf.contrib.layers.flatten(conv2)
        fc1 = tf.layers.dense(fc1, 1024)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        # Output with 10 nerons, each stands for the probability of a number
        out = tf.layers.dense(fc1, n_classes)
        
    return out
        

In [ ]:
# build the whole train/eval/predict elements together
def model_fn(features, labels, mode):
    logits_train = conv_net(features, n_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, n_classes, dropout, reuse=True, is_training=False)
    
    pred_classes = tf.argmax(logits_test, axis =1) # axis=1 means max in row
    pred_probas = tf.nn.softmax(logits_test)
    
    # predict
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
     # train
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train, labels =tf.cast(labels, dtype=tf.int32)))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # evaluate
    acc_op = tf.metrics.accuracy(labels= labels, predictions = pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs
    
 

In [ ]:
model = tf.estimator.Estimator(model_fn)

## Define input function for training

In [ ]:
input_fn = tf.estimator.inputs.numpy_input_fn(
x = {'images':minist.train.images}, y=minist.train.labels,
batch_size = batch_size, num_epochs=None, shuffle=True)

# train model
model.train(input_fn, steps = num_steps)

## Define input function for evalutation

In [ ]:
input_fn = tf.estimator.inputs.numpy_input_fn(
x= {'images':minist.test.images}, y = minist.test.labels,
   batch_size = batch_size, shuffle = False)
model.evaluate(input_fn)

## Prediction single images

In [ ]:
n_images = 4
test_images = minist.test.images[:n_images]
input_fn = tf.estimator.inputs.numpy_input_fn(
x= {'images':test_images}, shuffle = False)
preds = list(model.predict(input_fn))

for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap = 'gray')
    plt.show()
    print("Prediction label: {}".format(preds[i]))